In [2]:
import subprocess
from pybedtools import BedTool
import os
os.environ['MPLBACKEND'] = 'Agg'
os.chdir("/zata/zippy/ramirezc/splice-model-benchmark/golden_standard")

exons = BedTool('forward_exons.bed')
merged_exons = exons.merge()
sorted_exons = merged_exons.sort(g='genome.sizes')
non_exonic = sorted_exons.complement(g='genome.sizes')
non_exonic.saveas('blacklist.bed')

cmd = [
    "bamCoverage",
    "--bam", 'downloads/ENCBS020DQC/ENCFF099FKC.bam',
    "--outFileName", 'coverage.bg',
    "--outFileFormat", "bedgraph",
    "--filterRNAstrand", "forward",
    "--binSize", "10",
    "--blackListFileName", 'blacklist.bed',
    "--normalizeUsing", "RPKM",
    "--numberOfProcessors", '16'
]
subprocess.run(cmd, check=True)

There are 70184575 alignments, of which 53567609 are completely within a blacklist region.
Due to filtering, 95.39873767829457% of the aforementioned alignments will be used 15852375.804431397


normalization: RPKM
bamFilesList: ['downloads/ENCBS020DQC/ENCFF099FKC.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 16
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 10
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000
filter_strand: forward
Offset: [1, -1]


CompletedProcess(args=['bamCoverage', '--bam', 'downloads/ENCBS020DQC/ENCFF099FKC.bam', '--outFileName', 'coverage.bg', '--outFileFormat', 'bedgraph', '--filterRNAstrand', 'forward', '--binSize', '10', '--blackListFileName', 'blacklist.bed', '--normalizeUsing', 'RPKM', '--numberOfProcessors', '16'], returncode=0)